In [35]:
import re
import os
import string
from nltk import sent_tokenize, word_tokenize, download
from IPython.display import clear_output
from collections import Counter
import gc
import pickle
"""
# Save objects
with open('object_name', 'wb') as f:
    pickle.dump(object_variable, f)

# Load objects
with open('object_name', 'rb') as f:
    object_variable = pickle.load(f)
"""
print("Imports Completed")

Imports Completed


In [36]:
download('punkt')

[nltk_data] Downloading package punkt to /home/datum/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
def clean_text(text):
    """ 
    1. Remove html like text from europarl e.g. <Chapter 1>
    2. Remove line breaks
    3. Reduce all whitespaces to 1
    4. turn everything to lower case
    """
    clean = re.compile('<.*?>')
    
    out = text.replace('\n', ' ') # Remove line breaks
    out = re.sub(clean, ' ', out) # Remove tagged text e.g. <Chapter 1>
    out = re.sub(' +', ' ', out) # Reduce whitespace down to one
    
    out = out.lower() # Turn everything to lower case
    
    return out

In [39]:
abs_path = os.getcwd()
path = abs_path + '/en/'
text = ''
corpus_clean = []
sentences = []
corpus_original = ''
total = len(os.listdir(path)) # Total files
count = 0

for file in os.listdir(path):
    f = open(path + file, 'r', encoding="utf-8")
    file_text = f.read()
    corpus_original = corpus_original + file_text
    f.close()

    regex = re.compile('[%s]' % re.escape(string.punctuation))
    file_sentences = [regex.sub('', sent).strip() for sent in sent_tokenize(clean_text(file_text))]

    corpus_clean = corpus_clean + file_sentences
    count += 1
    
    clear_output(wait = True)
    print('File ' + file + ' finished. Completed ' + str(round(count*100/total,2)) + '%')

File ep-99-04-14.txt finished. Completed 100.0%


In [ ]:
"""
demo corpus file:
------------------------------------------------------------------
original text: <class 'str'>
<CHAPTER ID="005">
Voting time
<SPEAKER ID="104" NAME="President">
The next item is the vote.
<P>
(For the results and other details on the vote: see Minutes)
------------------------------------------------------------------
Its clean_text(text): <class 'str'>
 voting time the next item is the vote. (for the results and other details on the vote: see minutes) 
------------------------------------------------------------------
Its sent_tokenize(clean_text(text)): <class 'list'>
[' voting time the next item is the vote.', '(for the results and other details on the vote: see minutes)']
------------------------------------------------------------------
Its sentences = [sent.strip() for sent in sent_tokenize(clean_text(text))]: <class 'list'>
['voting time the next item is the vote.', '(for the results and other details on the vote: see minutes)']
------------------------------------------------------------------
"""

In [40]:
# Save the basic objects:
with open('corpus_original', 'wb') as f:
    pickle.dump(corpus_original, f)
with open('corpus_clean', 'wb') as f:
    pickle.dump(corpus_clean, f)

In [41]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
corpus_clean_string = regex.sub('', clean_text(corpus_original))

print('-------------------------')
print('corpus_clean_string created.')
print('-------------------------')

with open('corpus_clean_string', 'wb') as f:
    pickle.dump(corpus_clean_string, f)

del corpus_original

-------------------------
corpus_clean_string created.
-------------------------


In [42]:
gc.collect()

317

In [44]:
AllWords = word_tokenize(corpus_clean_string)
print('-------------------------')
print('Words Tokenized.')
print('-------------------------')

vocabulary = set(AllWords)
print('Vocabulary Created.')
print('-------------------------')

WordCounts = Counter(AllWords)
print('WordCounts Calculated.')
print('-------------------------')

with open('AllWords', 'wb') as f:
    pickle.dump(AllWords, f) 

with open('vocabulary', 'wb') as f:
    pickle.dump(vocabulary, f) 
    
with open('WordCounts', 'wb') as f:
    pickle.dump(WordCounts, f) 

del corpus_clean_string, AllWords

-------------------------
Words Tokenized.
-------------------------
Vocabulary Created.
-------------------------
WordCounts Calculated.
-------------------------


In [46]:
gc.collect()

0

#### Early starting point: Instead of all the above..

In [47]:
corpus_clean = None
WordCounts = None
vocabulary = None

# Load objects

with open('corpus_clean', 'rb') as f:
    corpus_clean = pickle.load(f)

with open('vocabulary', 'rb') as f:
    vocabulary = pickle.load(f)
    
with open('WordCounts', 'rb') as f:
    WordCounts = pickle.load(f)

In [48]:
# Ignore low frequency words
valid_vocabulary = [k for k,v in WordCounts.items() if v > 10]
invalid_vocabulary = [k for k,v in WordCounts.items() if v <= 10]
print("valid voc", len(valid_vocabulary))
print("invalid voc", len(invalid_vocabulary))

with open('valid_vocabulary', 'wb') as f:
    pickle.dump(valid_vocabulary, f) 
    
with open('invalid_vocabulary', 'wb') as f:
    pickle.dump(invalid_vocabulary, f) 

valid voc 32678
invalid voc 109653


In [60]:
# Replace OOV words in sentences
total = len(corpus_clean)
for i in range(0,len(corpus_clean)):
    for word in valid_vocabulary:
        corpus_clean[i].replace(word, 'UNK')
    clear_output(wait = True)
    print('Sentences processed ' + str(i+1) + ' out of ' + str(total) )

Sentences processed 393 out of 2217535


KeyboardInterrupt: 

In [ ]:
corpus_clean_no_OOV = None
with open('corpus_clean_no_OOV', 'rb') as f:
    corpus_no_OOV = pickle.load(f)